In [1]:
from __future__ import division, print_function
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
from importlib import reload  # Python 3
import utils_tri; reload(utils_tri)
from utils_tri import *

Using TensorFlow backend.


In [2]:
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from utils_tri import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [12]:
import vgg16_tri; reload(vgg16_tri)
from vgg16_tri import Vgg16

In [13]:
path = 'data/dogscats/sample/'

#### Tasks

- Load sample dogs and cats data.
- Finetune VGG and train the last layer
- Run on the full dataset (later)
- Practice plotting images
- Plot confusion matrix


In [14]:
# load training data
# we can either get data or get batches
train_batches = get_batches(path+'train')
valid_batches = get_batches(path+'valid')

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [19]:
# load VGG 16 model
vgg = Vgg16()

In [20]:
train_batches.class_indices

{'cats': 0, 'dogs': 1}

In [21]:
vgg.finetune(train_batches)

In [22]:
vgg.fit(train_batches, valid_batches, batch_size=4, nb_epoch=4)

Epoch 1/4
 8/40 [=====>........................] - ETA: 267s - loss: 1.9546 - acc: 0.4688

KeyboardInterrupt: 